In [3]:
import requests
from bs4 import BeautifulSoup as bs

In [4]:
year = "1995"
url =f'https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/{year}'

dataset_response = requests.get(url)
name_list = [i.text for i in bs(dataset_response.content).find_all('td') if i.text[-3:] =='cdf']

print(f" Year {year} has {len(name_list)} dataset")


 Year 1995 has 365 dataset


In [ ]:
import requests
from concurrent.futures import ThreadPoolExecutor
import math

MAX_RETRIES = 3  
def download_file(name):
    retries = 0
    while retries < MAX_RETRIES:
        try:
            response = requests.get(f'https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/{year}/{name}')
            if response.status_code == 200:
                with open(f'download\\{year}\\{name}', 'wb') as fp:
                    fp.write(response.content)
                print(f"Downloaded: {name}")

                break
            else:
                print(f"Error downloading {name}: Got status code {response.status_code}")
        except Exception as e:
            print(f"Error downloading {name}: {e}")
        
        retries += 1
        print(f"Retrying ({retries}/{MAX_RETRIES})...")
    


progress = len(name_list)
def download_files_with_threadpool(name_list, max_workers=10):
    chunk_size = math.ceil(len(name_list) / max_workers)
    chunks = [name_list[i:i+chunk_size] for i in range(0, len(name_list), chunk_size)]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for chunk in chunks:
            executor.map(download_file, chunk)


download_files_with_threadpool(name_list, max_workers=10)


In [ ]:
# import requests
# from concurrent.futures import ThreadPoolExecutor



# def download_file(name):
#     try:
#         response = requests.get(f'https://cdaweb.gsfc.nasa.gov/pub/data/wind/swe/swe_h1/{year}/{name}')
#         with open(f'download\\{year}\\{name}', 'wb') as fp:
#             fp.write(response.content)
        
#         print(f"Downloaded: {name}")
#     except Exception as e:
#         print(f"Error downloading {name}: {e}")

# def download_files_with_threadpool(name_list):
#     with ThreadPoolExecutor() as executor:
#         executor.map(download_file, name_list)

# download_files_with_threadpool(name_list)


In [6]:
#combine

import os
from modules.combine_cdf import *
import cdflib
import spacepy.pycdf as pycdf


def make_cdf(path,combined_cdf):

    cdf_file_combined = pycdf.CDF(path, '')


    for var_name, var_data in combined_cdf.items():
        cdf_file_combined[var_name] = var_data

    cdf_file_combined.close()

cyear = '2003'
lisdir = os.listdir(f'download//{cyear}')
for cdfs in range(len(lisdir)):
  
    cdf1,cdf2,cdf3 = cdflib.CDF(f'download//{cyear}/{lisdir[cdfs]}'),cdflib.CDF(f'download//{cyear}/{lisdir[cdfs+1]}'),cdflib.CDF(f'download//{cyear}/{lisdir[cdfs+2]}')
    path = f"download//combined//{cyear}/wi_h1_swe_{lisdir[cdfs].split("_")[3]} - {lisdir[cdfs+2].split("_")[3]}.cdf"
    
    # print(len(cdf1['BX']),len(cdf2['BX']),len(cdf3['BX']))
    result_cdf = merge_cdf(cdf1,cdf2,cdf3)
    combined_cdf = make_cdf(path,result_cdf)

    if cdfs+2 ==len(lisdir)-1:
        break
    
 

        

# testing plot

In [18]:
import cdflib
from modules.calparam import *
import os
from modules.plotter import Plotting
import matplotlib.pyplot as plt
import concurrent.futures


plt.ioff()
def process_file(cdf_file_name):
    plt_name = cdf_file_name.split("_")[-1].split(".")[0]
    plt_ad = 'download//PLOT//2003'
    print(plt_name)
    cdf_file = cdflib.CDF(f'download//combined//2003/{cdf_file_name}')

    time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))
    BX,BY,BZ,b_mag = bmagnitude(cdf_file)
    p_density = thresold(cdf_file.varget('Proton_Np_moment'),1e4)
    vp = Vp_vec(cdf_file)
    temp = Temperature(cdf_file)
    p_beta = plasma_beta(p_density,b_mag,temp)
    tehta_angle,phi_angle = angels(BX,BY,BZ,b_mag)

    plot  = Plotting(plt_name,plt_ad)

    plot.ipplot(time_range,b_mag,BX,BY,BZ,tehta_angle,phi_angle,vp,p_density,temp,p_beta)

lis_dir = os.listdir(r'download\combined\2003')

# Use ThreadPoolExecutor to parallelize processing and plotting tasks
# with concurrent.futures.ThreadPoolExecutor(10) as executor:
#     executor.map(process_file, lis_dir)
# lis_dir
for cdf_file_path in lis_dir:
    process_file(cdf_file_path)

20030101 - 20030103
20030102 - 20030104
20030103 - 20030105
20030104 - 20030106
20030105 - 20030107
20030106 - 20030108
20030107 - 20030109
20030108 - 20030110
20030109 - 20030111
20030110 - 20030112


In [9]:

# from modules.plotter import Plotting
# cdf_file = cdflib.CDF(r'download\\2003\\wi_h1_swe_20031021_v01.cdf')

# time_range= cdflib.cdfepoch.to_datetime(cdf_file.varget('epoch'))
# BX,BY,BZ,b_mag = bmagnitude(cdf_file)
# p_density = thresold(cdf_file.varget('Proton_Np_moment'),1e4)
# vp = Vp_vec(cdf_file)
# temp = Temperature(cdf_file)
# p_beta = plasma_beta(p_density,b_mag,temp)
# tehta_angle,phi_angle = angels(BX,BY,BZ,b_mag)
# plot  = Plotting("test")
# plot.ipplot("test",time_range,b_mag,BX,BY,BZ,tehta_angle,phi_angle,vp,p_density,temp,p_beta)